In [ ]:
import requests
from pprint import pprint
url_api = f"https://gateway.chotot.com/v1/public/recommender/homepage?fingerprint=b7294700-f5eb-46d3-9c65-f95f86c813f3&page=1"
r = requests.get(url_api)
pprint(r.text)



In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_stdlib_context

In [1]:
config = {
    'phone_number': '0366333953', # phone number
    'password': 'Vi21082004@1', # password
    'delay_time': 10,
    'base_url': "https://shopee.vn",
    'brand_url': 'Máy-Tính-Laptop-cat.11035954'
}

In [3]:
options = webdriver.ChromeOptions() 
options.headless = False
driver = uc.Chrome(options=options)
driver.maximize_window()

In [5]:
# hàm để cuộn trang
def scroll_down(pct=0.55, time_sleep=1):
    height = driver.execute_script("return document.body.scrollHeight")

    height = int(height*pct)
    for i in range(0, height, 300):
        driver.execute_script("window.scrollTo({}, {});".format(i, i+300))
        sleep(time_sleep)

In [ ]:
driver.get(config["base_url"])
# WebDriverWait(driver, config['delay_time']).until(EC.presence_of_element_located((By.XPATH, '//button[@class="_66HDIC"]')))
# driver.save_screenshot("login.png")
# driver.find_element(By.XPATH, '//button[@class="_66HDIC"]').click()
# đợi trong 10s nếu tìm thấy XPATH thì tiếp tục
WebDriverWait(driver, config['delay_time']).until(EC.presence_of_element_located((By.XPATH, '//input[@type="text"]')))

# nhập tk và mk để login
driver.find_element(By.XPATH,'//input[@type="text"]').send_keys(config['phone_number'])
sleep(5)
driver.find_element(By.XPATH,'//input[@type="password"]').send_keys(config['password'])
sleep(5)
driver.find_element(By.XPATH,'//button[@class="b5aVaf PVSuiZ Gqupku qz7ctP qxS7lQ Q4KP5g"]').click()

# đợi 10s nếu url hiện tại sau dấu ? có chứa "is_from_login=true" thì tiếp tục
try:
    WebDriverWait(driver, config['delay_time']).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="shopee-avatar__placeholder"]')))
    print("Login success")
except:
    print("Login failed")
sleep(1)
driver.save_screenshot("login.png")

In [9]:
driver.get(f"{config['base_url']}/{config['brand_url']}")

In [ ]:
scroll_down()

In [ ]:
box = driver.find_element(By.XPATH, '//li[@data-sqe="item"]')
link_image = box.find_element(By.XPATH, '//div[@class="Vd2yUl _8wgpQO"]//img').get_attribute('src')
link_product = box.find_element(By.XPATH, '//a[@data-sqe="link"]').get_attribute('href')
print(f'link ảnh: {link_image}\nlink sản phẩm: {link_product}')

In [ ]:
driver.get(link_product)
sleep(1)
scroll_down(0.2)
soup = BeautifulSoup(driver.page_source)

In [ ]:
def get_data(soup, selector_string):
    try:
        data = soup.select(selector_string)
        return data[0].text
    except:
        return None

def info_shop(soup):
    res = {'Đánh giá': None,
            'tỉ lệ phản hồi': None,
            'tham gia': None,
            'Sản phẩm': None,
            'thời gian phản hồi': None,
            'Người theo dõi': None}
    res['shop_name'] = get_data(soup, ".fV3TIn")
    brand = get_data(soup, ".ZUZ1FO") # shop yêu thích
    if not brand:
        brand = 'mall' if get_data(soup, ".official-shop-new-badge")=='' else 'normal' # shop mall
    res['shop_brand'] = brand
    try:
        for i in soup.select('.NGzCXN > .YnZi6x'):
            try:
                res[i.label.text] = i.span.text
            except:
                pass
    except:
                pass
    return res

def extract_data(soup) -> dict:
    '''hàm này để trích xuất dữ liệu'''
    res = {}
    # lấy tên sản phẩm
    res['name_product'] = get_data(soup, ".WBVL_7 > span") #soup.find(class_="WBVL_7").span.text
    # lấy giá
    res['price_origin'] = get_data(soup, ".qg2n76")
    res['price'] = get_data(soup, ".G27FPf")
    res['rate'] = get_data(soup, ".dQEiAI")
    res['review_count'] = get_data(soup, ".e2p50f > div[class='F9RHbS']")
    res['sale_count'] = get_data(soup, ".AcmPRb")
    res['like_count'] = get_data(soup, ".w2JMKY > .rhG6k7")
    res['fee_delivery'] = get_data(soup, ".PZGOkt")
    res['is_flash_sale'] = 1 if get_data(soup, ".x7M8PV") else 0
    res['quantity'] = get_data(soup, ".OaFP0p > div > div:nth-child(2)")
    res['descibe'] = get_data(soup, ".e8lZp3")
    res['detail'] = get_data(soup, ".Gf4Ro0")
    shop = info_shop(soup.find(class_="page-product__shop"))
    res.update(shop)
    return res

detail_data = extract_data(soup)
detail_data

In [ ]:
result = {
    "link_image": link_image,
    "link_product": link_product
}

result.update(detail_data)
result

In [ ]:
# chuyển json sang csv
df = pd.DataFrame([result]) #
df.to_csv("shopee.csv", mode='a', header=not pd.io.common.file_exists("shopee.csv"), index=False)

In [ ]:
def log(text):
    with open("log_error.txt", "a", encoding='utf-8') as f:
        f.write(text + "\n")

def get_data(soup, selector_string):
    try:
        data = soup.select(selector_string)
        return data[0].text
    except:
        return None

def info_shop(soup):
    res = {'Đánh giá': None,
            'tỉ lệ phản hồi': None,
            'tham gia': None,
            'Sản phẩm': None,
            'thời gian phản hồi': None,
            'Người theo dõi': None}
    res['shop_name'] = get_data(soup, ".fV3TIn")
    brand = get_data(soup, ".ZUZ1FO") # shop yêu thích
    if not brand:
        brand = 'mall' if get_data(soup, ".official-shop-new-badge")=='' else 'normal' # shop mall
    res['shop_brand'] = brand
    try:
        for i in soup.select('.NGzCXN > .YnZi6x'):
            try:
                res[i.label.text] = i.span.text
            except:
                pass
    except:
                pass
    return res

def extract_data(soup) -> dict:
    '''hàm này để trích xuất dữ liệu'''
    res = {}
    # lấy tên sản phẩm
    res['product_name'] = get_data(soup, ".WBVL_7 > span") #soup.find(class_="WBVL_7").span.text
    # lấy giá
    res['price_origin'] = get_data(soup, ".qg2n76")
    res['price'] = get_data(soup, ".G27FPf")
    res['rate'] = get_data(soup, ".dQEiAI")
    res['review_count'] = get_data(soup, ".e2p50f > div[class='F9RHbS']")
    res['sale_count'] = get_data(soup, ".AcmPRb")
    res['like_count'] = get_data(soup, ".w2JMKY > .rhG6k7")
    res['fee_delivery'] = get_data(soup, ".PZGOkt")
    res['is_flash_sale'] = 1 if get_data(soup, ".x7M8PV") else 0
    res['quantity'] = get_data(soup, ".OaFP0p > div > div:nth-child(2)")
    res['descibe'] = get_data(soup, ".e8lZp3")
    res['detail'] = get_data(soup, ".Gf4Ro0")
    shop = info_shop(soup.find(class_="page-product__shop"))
    res.update(shop)
    return res

In [ ]:
n_page = 10
save_file = 'shopee.csv'
for page in range(n_page):
    log(f'# {page}')
    url_page = f"{config['base_url']}/{config['brand_url']}?page={page}"
    driver.get(url_page)
    sleep(8)
    scroll_down(0.55)
    boxs = driver.find_elements(By.XPATH, '//a[@data-sqe="link"]')
    link_images = [box.find_element(By.XPATH, '//div[@class="Vd2yUl _8wgpQO"]//img').get_attribute('src') for box in boxs]
    link_products = [box.get_attribute('href') for box in boxs]
    loop = tqdm(link_products, desc=f"Page {page}", ncols=100, colour='green') # tạo thanh tiến trình cho đẹp thôi nha :>>
    for idx, link_product in enumerate(loop):
        driver.get(link_product)
        sleep(8)
        scroll_down(0.8)
        soup = BeautifulSoup(driver.page_source)
        detail_data = extract_data(soup)
        result = {
            "link_image": link_images[idx],
            "link_product": link_products[idx],
        }
        result.update(detail_data)
        df = pd.DataFrame([result])
        df.to_csv(save_file, mode='a', header=not pd.io.common.file_exists(save_file), index=False)
        check = df['product_name'][0]
        if check==None:
            log(f'{idx}')

In [ ]:
driver.quit()

In [ ]:
df = pd.read_csv("shopee.csv")
df.drop_duplicates(subset=['product_name'], inplace=True)
# df.reset_index(inplace=True, drop=True)
# df.shape

In [ ]:
df.head()